### Air Quality Monitor and Comparison

1. Dataset

Name: Air Quality Data - Updated Hourly
This website provides global air quality data updated hourly on the pollutants of PM2.5, PM10, O3 (Ozone), NO2 (Nitrogen dioxide), SO2 (Sulfur dioxide), CO (Carbon monoxide)

2. Research Questions

How do hourly air pollution patterns (PM₂.₅, NO₂, O₃) vary across major cities, and when do cities experience statistically significant spikes relative to their typical baseline?
We will focus on 6 major cities that allows us to have strong diversity for meaningful comparison across the globe
NYC
London
Hong Kong
Delhi
São Paulo
Tokyo

3. Notebook Link

4. Target Visualization
To be updated
5. Known Unknowns

Known:
Timestamped hourly pollutant readings for (PM2.5, NO2, O3), which allows us to conduct pattern analysis and produce day-of week comparisons, spike detection, and form rolling averages
All pollutants share the same measurement of µg/m³
Geographic coordinates, which allows us to monitor stations for cross-city comparisons, spatial clustering, and mapping in Streamlit
Clearly specified pollutant types allows us to compare traffic-related and climate-related patterns
Real-time data enables a live dashboard for trend and monitoring applications in our application development

Unknown:
Data Quality may vary as OpenAQ aggregates from multiple providers, such as government monitors, low-cost sensors, and private contributors. We are unsure if all monitors are calibrated equally, which may reflect monitoring quality difference when using the data for cross-city comparisons
Station placement bias because the stations are not randomly distributed, as they may be placed near highways, industrial zones, residential areas, etc. that may not represent the entire city fairly. This affects spike interpretation and baseline calculation. We may need to consider pulling data from multiple locations within a city with city-level aggregates, using median values and standard deviations to make the cities more comparable
The definition of spike needs to be constructed
No direct causal variables of traffic counts, weather (temperature and wind), policy changes, industrial output, wildfire data. We may need another set of data(s) to identify the direct causes

6. Anticipated Challenges

Making sure the time stamps are consistent across the data and making sure the time zones are correct
Computing the baseline and spikes correctly, by using city-level hourly aggregation, rolling baseline, and rolling variability estimates. This requires computing the z-score or robust z-score
Need to code to log data pull times and timestamps of a specific time to produce live data on our application


## To be Updated

In [19]:
from dotenv import load_dotenv
import os, time, requests
import pandas as pd
from datetime import datetime, timedelta, timezone
import re

# ----------------------------
# 0) Config
# ----------------------------
load_dotenv()
API_KEY = os.getenv("OPENAQ_API_KEY")
if not API_KEY:
    raise ValueError("Missing OPENAQ_API_KEY (check .env)")

HEADERS = {"X-API-Key": API_KEY}
BASE = "https://api.openaq.org/v3"

CITY = "NYC"
BBOX = "-74.30,40.50,-73.70,40.92"
PARAMETERS = ["pm25", "no2", "o3"]

now = datetime.now(timezone.utc)
start = now - timedelta(hours=24)   # 需要更保险可改 days=7
DATE_FROM = start.isoformat(timespec="seconds")
DATE_TO = now.isoformat(timespec="seconds")

LOC_LIMIT = 60
SENSOR_LIMIT = 200
HOURS_LIMIT = 200
SLEEP_S = 0.12


# ----------------------------
# 1) HTTP helper
# ----------------------------
def get_json(url, params=None):
    r = requests.get(url, headers=HEADERS, params=params, timeout=30)
    r.raise_for_status()
    time.sleep(SLEEP_S)
    return r.json()


def normalize_param(p):
    if p is None:
        return ""
    if isinstance(p, str):
        return p.lower()
    if isinstance(p, dict):
        return str(p.get("name") or p.get("code") or "").lower()
    return str(p).lower()


# ----------------------------
# 2) Recursive leaf collection (key part)
# ----------------------------
ISO_HINT = re.compile(r"\d{4}-\d{2}-\d{2}T")  # quick ISO8601 hint

def collect_leaves(obj, path=""):
    """
    Recursively collect leaf values with their paths.
    Returns list of tuples: (path, value)
    """
    out = []
    if isinstance(obj, dict):
        for k, v in obj.items():
            out.extend(collect_leaves(v, f"{path}.{k}" if path else str(k)))
    elif isinstance(obj, list):
        for i, v in enumerate(obj):
            out.extend(collect_leaves(v, f"{path}[{i}]"))
    else:
        out.append((path, obj))
    return out


def pick_datetime_from_record(rec):
    """
    Try common paths first, then fallback to scanning all string leaves that look like ISO datetime.
    """
    # common candidates (your v3 may differ)
    candidates = [
        rec.get("datetime"),
        (rec.get("datetime") or {}).get("utc") if isinstance(rec.get("datetime"), dict) else None,
        (rec.get("date") or {}).get("utc") if isinstance(rec.get("date"), dict) else None,
        (rec.get("period") or {}).get("datetimeFrom") if isinstance(rec.get("period"), dict) else None,
        (rec.get("period") or {}).get("datetimeTo") if isinstance(rec.get("period"), dict) else None,
        rec.get("timestamp"),
        rec.get("time"),
    ]
    for c in candidates:
        if isinstance(c, str) and ISO_HINT.search(c):
            dt = pd.to_datetime(c, errors="coerce", utc=True)
            if not pd.isna(dt):
                return c

    # fallback: scan all leaves
    leaves = collect_leaves(rec)
    for p, v in leaves:
        if isinstance(v, str) and ISO_HINT.search(v):
            dt = pd.to_datetime(v, errors="coerce", utc=True)
            if not pd.isna(dt):
                return v

    return None


def pick_value_from_record(rec):
    """
    Try common paths first, then fallback to first numeric leaf.
    """
    # common candidates
    candidates = [
        rec.get("value"),
        rec.get("avg"),
        rec.get("mean"),
        rec.get("median"),
        rec.get("average"),
    ]
    for c in candidates:
        if isinstance(c, (int, float)):
            return c
        if isinstance(c, dict):
            for k in ["value", "avg", "mean", "median"]:
                if isinstance(c.get(k), (int, float)):
                    return c.get(k)

    # fallback: scan numeric leaves
    leaves = collect_leaves(rec)
    for p, v in leaves:
        if isinstance(v, (int, float)):
            return v

    return None


def pick_unit_from_record(rec):
    u = rec.get("unit")
    if isinstance(u, str):
        return u
    p = rec.get("parameter")
    if isinstance(p, dict) and isinstance(p.get("unit"), str):
        return p.get("unit")

    # fallback: scan for likely unit strings (optional)
    leaves = collect_leaves(rec)
    for path, v in leaves:
        if isinstance(v, str) and v in ["µg/m³", "ug/m3", "ppm", "ppb"]:
            return v
    return None


# ----------------------------
# 3) OpenAQ data functions
# ----------------------------
def get_locations_by_bbox(bbox, limit=60):
    data = get_json(f"{BASE}/locations", params={"bbox": bbox, "limit": limit, "page": 1})
    return data.get("results", [])


def get_sensors_for_location(location_id, limit=200):
    data = get_json(f"{BASE}/locations/{location_id}/sensors", params={"limit": limit, "page": 1})
    return data.get("results", [])


def get_sensor_hours(sensor_id, date_from, date_to, limit=200):
    data = get_json(
        f"{BASE}/sensors/{sensor_id}/hours",
        params={"date_from": date_from, "date_to": date_to, "limit": limit, "page": 1}
    )
    return data.get("results", [])


def find_working_sensor_for_param(locs, param):
    """
    Find first (location_id, sensor_id, hours_records) that:
    - has matching parameter sensor
    - hours endpoint returns non-empty list
    """
    target = param.lower()
    for loc in locs:
        loc_id = loc["id"]
        sensors = get_sensors_for_location(loc_id, limit=SENSOR_LIMIT)
        matched = [s for s in sensors if normalize_param(s.get("parameter")) == target]
        for s in matched:
            sid = s["id"]
            hours = get_sensor_hours(sid, DATE_FROM, DATE_TO, limit=HOURS_LIMIT)
            if hours:
                return loc_id, sid, hours
    return None, None, []


# ----------------------------
# 4) Main: build sample df
# ----------------------------
locs = get_locations_by_bbox(BBOX, limit=LOC_LIMIT)
print(f"{CITY} locations returned:", len(locs))
if not locs:
    raise RuntimeError("No locations in bbox. Try enlarging bbox.")

rows = []
debug_first_record = None

for param in PARAMETERS:
    loc_id, sensor_id, hours = find_working_sensor_for_param(locs, param)
    if not sensor_id:
        print(f"[WARN] No working sensor for {param}. Try wider window (days=7).")
        continue

    print(f"{param}: chosen location_id={loc_id}, sensor_id={sensor_id}, hours returned={len(hours)}")

    if hours and debug_first_record is None:
        debug_first_record = hours[0]

    for rec in hours:
        dt = pick_datetime_from_record(rec)
        val = pick_value_from_record(rec)
        unit = pick_unit_from_record(rec)

        rows.append({
            "city": CITY,
            "parameter": param,
            "location_id": loc_id,
            "sensor_id": sensor_id,
            "datetime": dt,
            "value": val,
            "unit": unit
        })

df = pd.DataFrame(rows)
print("raw df shape:", df.shape)

# quick debug: show one raw hour record structure if cleaning still fails
if debug_first_record is not None:
    print("sample hour record (for debugging):", debug_first_record)

if df.empty:
    print("No rows collected at all.")
else:
    # parse + clean (now should keep rows)
    df["datetime"] = pd.to_datetime(df["datetime"], errors="coerce", utc=True)
    df["value"] = pd.to_numeric(df["value"], errors="coerce")

    # print missing rates before dropping
    print("missing datetime rate:", df["datetime"].isna().mean())
    print("missing value rate:", df["value"].isna().mean())

    df_clean = df.dropna(subset=["datetime", "value"]).sort_values(["parameter", "datetime"])
    print("clean df shape:", df_clean.shape)
    display(df_clean.head(20))


NYC locations returned: 60
pm25: chosen location_id=384, sensor_id=673, hours returned=200
no2: chosen location_id=853, sensor_id=1523, hours returned=200
o3: chosen location_id=384, sensor_id=671, hours returned=200
raw df shape: (600, 7)
sample hour record (for debugging): {'value': 2.4, 'flagInfo': {'hasFlags': False}, 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': None}, 'period': {'label': '1hour', 'interval': '01:00:00', 'datetimeFrom': {'utc': '2016-03-12T08:00:00Z', 'local': '2016-03-12T03:00:00-05:00'}, 'datetimeTo': {'utc': '2016-03-12T09:00:00Z', 'local': '2016-03-12T04:00:00-05:00'}}, 'coordinates': None, 'summary': {'min': 2.4, 'q02': 2.4, 'q25': 2.4, 'median': 2.4, 'q75': 2.4, 'q98': 2.4, 'max': 2.4, 'avg': 2.4, 'sd': None}, 'coverage': {'expectedCount': 1, 'expectedInterval': '01:00:00', 'observedCount': 1, 'observedInterval': '01:00:00', 'percentComplete': 100.0, 'percentCoverage': 100.0, 'datetimeFrom': {'utc': '2016-03-12T08:00:00Z', 'local': 

,city,parameter,location_id,sensor_id,datetime,value,unit
200,NYC,no2,853,1523,2016-03-31 18:00:00+00:00,0.009,ppm
201,NYC,no2,853,1523,2016-03-31 19:00:00+00:00,0.011,ppm
202,NYC,no2,853,1523,2016-04-01 02:00:00+00:00,0.010,ppm
203,NYC,no2,853,1523,2016-04-01 03:00:00+00:00,0.009,ppm
204,NYC,no2,853,1523,2016-04-01 04:00:00+00:00,0.004,ppm
205,NYC,no2,853,1523,2016-04-01 05:00:00+00:00,0.003,ppm
206,NYC,no2,853,1523,2016-04-01 06:00:00+00:00,0.003,ppm
207,NYC,no2,853,1523,2016-04-01 07:00:00+00:00,0.004,ppm
208,NYC,no2,853,1523,2016-04-01 08:00:00+00:00,0.004,ppm
209,NYC,no2,853,1523,2016-04-01 09:00:00+00:00,0.007,ppm
